In [ ]:
import seaborn as sns
sns.set_theme(font='Liberation Serif',
              rc={'figure.figsize': (7.5,3.75),
                  'font.size': 11,
                 })

import numpy as np
import pandas as pd
import re

# Huggingface
from transformers import AutoModel, AutoTokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
model = AutoModel.from_pretrained(MODEL_NAME)

# PyTorch
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

In [ ]:
s = 'This is an example.'
s = tokenizer(s, return_tensors='pt')

In [3]:
with torch.no_grad():
    hidden_states = model(**s, output_hidden_states=True)['hidden_states']

## Indexing

hidden_states[i] == hidden states at i<sup>th</sup> layer of network.

hidden_states[i].shape == [num_examples, sequence_length, embedding_size]  

To get the 13 different embeddings for a single token position, we select the first and only example sentence, the position in the input sequence, and all values in the embedding matrix.  

In [ ]:
hidden_states[0][0,:,:]

In [ ]:
hidden_states[0]